### Consolidating Basic HDB Data 
In this notebook, we will be consolidating basic HDB data into one file. We will then be adding supplementary property information to the basic HDB resale dataframe.
Data Source: data.gov.sg
- Resale Flat Prices 1990 - 1999: https://beta.data.gov.sg/datasets/d_ebc5ab87086db484f88045b47411ebc5/view
- Resale Flat Prices 2000 - Feb 2012: https://beta.data.gov.sg/datasets/d_43f493c6c50d54243cc1eab0df142d6a/view
- Resale Flat Prices Mar 2012 to Dec 2014: https://beta.data.gov.sg/datasets/d_2d5ff9ea31397b66239f245f57751537/view
- Resale Flat Prices Jan 2015 to Dec 2016: https://beta.data.gov.sg/datasets/d_ea9ed51da2787afaf8e51f827c304208/view
- Resale flat prices Jan-2017 onwards: https://beta.data.gov.sg/datasets/d_8b84c4ee58e3cfc0ece0d773c8ca6abc/view
- HDB Property Information: https://beta.data.gov.sg/datasets/d_17f5382f26140b1fdae0ba2ef6239d2f/view

#### Load libraries

In [1]:
# Wrangling libraries
import numpy as np
import pandas as pd

# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Os Libraries
import os
import glob


In [6]:
# get working directory
path = os.getcwd()

# path to files
data_files = '/../04data/raw/HDB_resale/'
path = path + data_files

#check to see if works
os.listdir(path)


['ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv',
 'ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv',
 'ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv',
 'ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv',
 'ResaleFlatPricesBasedonApprovalDate19901999.csv']

In [7]:
# use glob to get all the csv files in the folder
csv_files = glob.glob(os.path.join(path, "*.csv"))
csv_files

['/Users/gobind/code/hgobinds/hdb-price-estimator/Gobind/../04data/raw/HDB_resale/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv',
 '/Users/gobind/code/hgobinds/hdb-price-estimator/Gobind/../04data/raw/HDB_resale/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv',
 '/Users/gobind/code/hgobinds/hdb-price-estimator/Gobind/../04data/raw/HDB_resale/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv',
 '/Users/gobind/code/hgobinds/hdb-price-estimator/Gobind/../04data/raw/HDB_resale/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv',
 '/Users/gobind/code/hgobinds/hdb-price-estimator/Gobind/../04data/raw/HDB_resale/ResaleFlatPricesBasedonApprovalDate19901999.csv']

In [8]:
# list of columns to keep
cols = ['month','town','flat_type','block','street_name','storey_range','floor_area_sqm','flat_model','lease_commence_date','resale_price']

# loop over the list of csv files, and concat them into one dataframe
df = pd.concat((pd.read_csv(f, usecols=cols) for f in csv_files), ignore_index=True)

# view 5 random rows of dataframe
df.sample(5)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
403816,2016-10,YISHUN,4 ROOM,443,YISHUN AVE 11,04 TO 06,92.0,Model A,2012,425000.0
446964,2018-11,SERANGOON,4 ROOM,332,SERANGOON AVE 3,04 TO 06,84.0,Simplified,1986,472000.0
128296,2003-09,PASIR RIS,4 ROOM,626,PASIR RIS DR 3,13 TO 15,104.0,Model A,1995,265000.0
675733,1993-11,QUEENSTOWN,5 ROOM,21,HOLLAND DR,07 TO 09,119.0,STANDARD,1975,315000.0
35946,2001-01,GEYLANG,EXECUTIVE,130,GEYLANG EAST AVE 1,16 TO 18,146.0,Maisonette,1987,485000.0


In [9]:
# View df column types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908041 entries, 0 to 908040
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                908041 non-null  object 
 1   town                 908041 non-null  object 
 2   flat_type            908041 non-null  object 
 3   block                908041 non-null  object 
 4   street_name          908041 non-null  object 
 5   storey_range         908041 non-null  object 
 6   floor_area_sqm       908041 non-null  float64
 7   flat_model           908041 non-null  object 
 8   lease_commence_date  908041 non-null  int64  
 9   resale_price         908041 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 69.3+ MB


In [10]:
# From the out above, there are no null values, df is complete
# Lets convert month column which consists of month and year into a date type
# Create new column of column date and drop unnecessary column
df["sold_year_month"] = pd.to_datetime(df["month"])
df.sort_values('sold_year_month')
df = df.drop(columns='month')


In [11]:
# View value counts and number unique for each column to see if anything is odd
for col in df.select_dtypes(include=object):
    print(f"Number of unique values in {col}: {df[col].nunique()}")
    print(f"{df[col].value_counts()} \n")


Number of unique values in town: 27
town
TAMPINES           78669
YISHUN             68794
BEDOK              65727
JURONG WEST        65350
WOODLANDS          64128
ANG MO KIO         51334
HOUGANG            49584
BUKIT BATOK        43241
CHOA CHU KANG      37443
BUKIT MERAH        33635
PASIR RIS          32760
SENGKANG           31065
TOA PAYOH          31029
QUEENSTOWN         28323
GEYLANG            27759
CLEMENTI           27648
BUKIT PANJANG      27122
KALLANG/WHAMPOA    26621
JURONG EAST        24404
SERANGOON          22511
BISHAN             21010
PUNGGOL            19239
SEMBAWANG          13137
MARINE PARADE       7948
CENTRAL AREA        7022
BUKIT TIMAH         2474
LIM CHU KANG          64
Name: count, dtype: int64 

Number of unique values in flat_type: 8
flat_type
4 ROOM              344041
3 ROOM              291283
5 ROOM              191121
EXECUTIVE            68491
2 ROOM               11270
1 ROOM                1297
MULTI GENERATION       279
MULTI-GENERATION 

In [12]:
# Flat type has two types of spelling - 'MULTI-GENERATION','MULTI GENERATION' lets change that
df['flat_type'] = df['flat_type'].str.replace('MULTI-GENERATION','MULTI GENERATION')
df['flat_type'].value_counts()


flat_type
4 ROOM              344041
3 ROOM              291283
5 ROOM              191121
EXECUTIVE            68491
2 ROOM               11270
1 ROOM                1297
MULTI GENERATION       538
Name: count, dtype: int64

In [13]:
# Flat model also seems to have many duplicates, lets change them
# mapping logic - https://sg.finance.yahoo.com/news/different-types-hdb-houses-call-020000642.html
correction_map = {'2-ROOM':'2-room',
                  'APARTMENT':'Apartment',
                  'Improved-Maisonette':'Executive Maisonette',
                  'IMPROVED-MAISONETTE':'Executive Maisonette',
                  'IMPROVED':'Improved',
                  'MAISONETTE':'Maisonette',
                  'Model A-Maisonette':'Maisonette',
                  'MODEL A-MAISONETTE':'Maisonette',
                  'MODEL A':'Model A',
                  'MULTI GENERATION':'Multi Generation',
                  'Premium Apartment Loft':'Premium Apartment',
                  'PREMIUM APARTMENT':'Premium Apartment',
                  'Premium Maisonette':'Executive Maisonette',
                  'SIMPLIFIED':'Simplified',
                  'STANDARD':'Standard',
                  'TERRACE':'Terrace',
                  'NEW GENERATION':'New Generation'}

df = df.replace({'flat_model': correction_map})

df['flat_model'].value_counts()


flat_model
Model A                 257407
Improved                237399
New Generation          186747
Simplified               56881
Premium Apartment        44931
Standard                 42110
Apartment                34898
Maisonette               31243
Model A2                  9920
DBSS                      3104
Adjoined flat             1223
Terrace                    686
Multi Generation           538
Type S1                    424
Executive Maisonette       210
Type S2                    209
2-room                      94
3Gen                        17
Name: count, dtype: int64

In [14]:
# Creating just a year sold column
df['sold_year'] = df['sold_year_month'].dt.strftime('%Y').astype('int')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908041 entries, 0 to 908040
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   town                 908041 non-null  object        
 1   flat_type            908041 non-null  object        
 2   block                908041 non-null  object        
 3   street_name          908041 non-null  object        
 4   storey_range         908041 non-null  object        
 5   floor_area_sqm       908041 non-null  float64       
 6   flat_model           908041 non-null  object        
 7   lease_commence_date  908041 non-null  int64         
 8   resale_price         908041 non-null  float64       
 9   sold_year_month      908041 non-null  datetime64[ns]
 10  sold_year            908041 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 76.2+ MB


In [15]:
# Derive number of years of lease remaining
df['sold_remaining_lease'] = 99 - (df['sold_year'] - df['lease_commence_date'])


In [16]:
# Derive number of years of lease remaining
df['remaining_lease_in_2023'] = 99 - (2023 - df['lease_commence_date'])


In [17]:
# View range of leases remaining in 2023
np.sort(df['remaining_lease_in_2023'].unique())

# oldest hdb has 42 years remaining.
# youngest is 98 which is weird since you need to wait 5 years before being able to sell
# could indicate data anomaly to investigate

array([42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58,
       59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75,
       76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
       93, 94, 95, 98])

In [20]:
# investigating the HDBs sold with 98 years lease remaining
df[df['remaining_lease_in_2023']==98]

# since there are only two, we will leave them in the dataset


,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,sold_year_month,sold_year,sold_remaining_lease,remaining_lease_in_2023
566726,QUEENSTOWN,3 ROOM,37,MARGARET DR,25 TO 27,62.0,Premium Apartment,2022,750000.0,2023-08-01,2023,98,98
568255,QUEENSTOWN,3 ROOM,37,MARGARET DR,31 TO 33,62.0,Premium Apartment,2022,768000.0,2023-09-01,2023,98,98


#### Remove Duplicates

In [22]:
df = df.drop_duplicates()


In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 906088 entries, 0 to 908040
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   town                     906088 non-null  object        
 1   flat_type                906088 non-null  object        
 2   block                    906088 non-null  object        
 3   street_name              906088 non-null  object        
 4   storey_range             906088 non-null  object        
 5   floor_area_sqm           906088 non-null  float64       
 6   flat_model               906088 non-null  object        
 7   lease_commence_date      906088 non-null  int64         
 8   resale_price             906088 non-null  float64       
 9   sold_year_month          906088 non-null  datetime64[ns]
 10  sold_year                906088 non-null  int64         
 11  sold_remaining_lease     906088 non-null  int64         
 12  remaining_lease_in_20

#### Save DF to File

In [37]:
df.to_parquet('../data/processed/HDB_resale_1990_2023.parquet.gzip',
              compression='gzip')


#### Load HDB Property Information Dataset

In [38]:
# load and view Property Information data
df_info = pd.read_csv('../04data/raw/HDBPropertyInformation.csv')
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12721 entries, 0 to 12720
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   blk_no                 12721 non-null  object
 1   street                 12721 non-null  object
 2   max_floor_lvl          12721 non-null  int64 
 3   year_completed         12721 non-null  int64 
 4   residential            12721 non-null  object
 5   commercial             12721 non-null  object
 6   market_hawker          12721 non-null  object
 7   miscellaneous          12721 non-null  object
 8   multistorey_carpark    12721 non-null  object
 9   precinct_pavilion      12721 non-null  object
 10  bldg_contract_town     12721 non-null  object
 11  total_dwelling_units   12721 non-null  int64 
 12  1room_sold             12721 non-null  int64 
 13  2room_sold             12721 non-null  int64 
 14  3room_sold             12721 non-null  int64 
 15  4room_sold         

#### Joining the DataFrames on Addresses

In [39]:
# Make standardised address columns for both DFs
df['address'] = df['block'] + " " + df['street_name']
df_info['address'] = df_info['blk_no'] + " " + df_info['street']

In [40]:
# Do a left join to merge the two data frames into one
# Since the resale df contains data of HDBs which do not exist anymore,
# we will be using the df with the property information for our left df since
# it only contains information from HDBs which still exist.
df_full = df.merge(df_info, how='left', on='address')

# Transpose and view df as it is too wide to view
df_full.head().T

,0,1,2,3,4
town,ANG MO KIO,KALLANG/WHAMPOA,KALLANG/WHAMPOA,KALLANG/WHAMPOA,KALLANG/WHAMPOA
flat_type,1 ROOM,3 ROOM,3 ROOM,3 ROOM,3 ROOM
block,309,44,20,14,46
street_name,ANG MO KIO AVE 1,BENDEMEER RD,ST. GEORGE'S RD,KG ARANG RD,OWEN RD
storey_range,10 TO 12,04 TO 06,04 TO 06,04 TO 06,01 TO 03
floor_area_sqm,31.0,63.0,67.0,103.0,68.0
flat_model,Improved,Standard,New Generation,New Generation,New Generation
lease_commence_date,1977,1981,1984,1984,1982
resale_price,9000.0,31400.0,66500.0,77000.0,58000.0
sold_year_month,1990-01-01 00:00:00,1990-01-01 00:00:00,1990-01-01 00:00:00,1990-01-01 00:00:00,1990-01-01 00:00:00


**Comments**
- There are some rows with no blk_no with no property information
- since there is no need for them, we can remove them 

In [41]:
# count of how many rows have no blk_no
len(df_full[df_full['blk_no'].isna()]['address'].unique())

190

In [42]:
# since only 190 rows out ~900K rows of data are affected, we will remove them
df_full = df_full[df_full['blk_no'].notna()]

In [43]:
# remove repeated columns and rental information
df_full = df_full.drop(columns=['blk_no','street','1room_rental','2room_rental','3room_rental','other_room_rental','bldg_contract_town'])


In [44]:
# view df
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 890376 entries, 1 to 906087
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   town                     890376 non-null  object        
 1   flat_type                890376 non-null  object        
 2   block                    890376 non-null  object        
 3   street_name              890376 non-null  object        
 4   storey_range             890376 non-null  object        
 5   floor_area_sqm           890376 non-null  float64       
 6   flat_model               890376 non-null  object        
 7   lease_commence_date      890376 non-null  int64         
 8   resale_price             890376 non-null  float64       
 9   sold_year_month          890376 non-null  datetime64[us]
 10  sold_year                890376 non-null  int64         
 11  sold_remaining_lease     890376 non-null  int64         
 12  remaining_lease_in_20

#### Save DataFrame to Parquet File

In [45]:
# Save df to file
#df_full.to_parquet('../data/processed/HDB_full_resale_info_1990_2023.parquet.gzip', compression='gzip')
